In [1]:
!pip install transformers librosa torch openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 42.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=0fb9303ce482d16fde009a285a1cbdab9b4c34095de7de335f5e4d4562c1505a
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [2]:

import torch
import librosa
import numpy as np
import whisper
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import warnings

warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:



ser_model_path = "/content/drive/MyDrive/dlProject/NModal/wav2vec2-ser-ravdess"

print("Loading SER Model (Wav2Vec2)...")
try:
    ser_processor = Wav2Vec2Processor.from_pretrained(ser_model_path)
    ser_model = Wav2Vec2ForSequenceClassification.from_pretrained(ser_model_path).to(device)
    print("✅ SER Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading SER model: {e}")

Loading SER Model (Wav2Vec2)...
✅ SER Model loaded successfully!


In [4]:
# Cell 5: Load your trained RoBERTa (ABSA) Model
from transformers import AutoTokenizer, AutoModelForTokenClassification

# PATH TO YOUR RO BERTA MODEL
absa_model_path = "/content/drive/MyDrive/dlProject/NModal/RoBERTa_ABSA_Final"

print("Loading ABSA Model (RoBERTa)...")
try:
    # ✅ CORRECT: Use AutoModelForTokenClassification for extracting specific words
    absa_tokenizer = AutoTokenizer.from_pretrained(absa_model_path)
    absa_model = AutoModelForTokenClassification.from_pretrained(absa_model_path).to(device)
    print("✅ ABSA Model loaded successfully as Token Classifier!")
except Exception as e:
    print(f"❌ Error loading ABSA model: {e}")

Loading ABSA Model (RoBERTa)...
✅ ABSA Model loaded successfully as Token Classifier!


In [5]:

print("Loading Whisper ASR model...")
asr_model = whisper.load_model("base")
print("✅ Whisper model loaded!")

Loading Whisper ASR model...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 184MiB/s]


✅ Whisper model loaded!


In [11]:
import torch
import librosa
import numpy as np

# 1. Define Labels
id2label = {0: "O", 1: "B-ASP", 2: "I-ASP"}

# 2. DEFINE YOUR ALLOWLIST (The "Food Domain" Filter)
# The pipeline will ONLY show aspects if they (or their singular form) are in this list.
VALID_RESTAURANT_TERMS = {
    # Food & Drink
    "food", "dish", "meal", "pizza", "pasta", "burger", "chicken", "meat",
    "steak", "soup", "salad", "sauce", "crust", "dessert", "cake", "ice cream",
    "drink", "wine", "beer", "coffee", "tea", "menu", "portion", "flavor", "taste",

    # Service & Staff
    "service", "staff", "waiter", "waitress", "server", "host", "manager", "chef",
    "delivery", "management",

    # Ambience & Place
    "ambience", "atmosphere", "place", "restaurant", "setting", "music", "lighting",
    "table", "seat", "seating", "view", "noise", "decor",

    # Money
    "price", "cost", "value", "bill", "check", "money"
}

def extract_aspects_custom(text, model, tokenizer):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2).cpu().numpy()[0]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    extracted_aspects = []
    current_aspect = []

    for token, label_id in zip(tokens, predictions):
        label = id2label[label_id]

        # Clean token
        clean_token = token.replace("Ġ", " ").replace("Ċ", "").strip()

        # Skip special tokens
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue

        if label == "B-ASP":
            if current_aspect:
                full_word = " ".join(current_aspect).strip()
                extracted_aspects.append(full_word)
            current_aspect = [clean_token]
        elif label == "I-ASP":
            current_aspect.append(clean_token)
        else:
            if current_aspect:
                full_word = " ".join(current_aspect).strip()
                extracted_aspects.append(full_word)
                current_aspect = []

    if current_aspect:
        full_word = " ".join(current_aspect).strip()
        extracted_aspects.append(full_word)

    # --- FILTERING STEP ---
    # Only keep aspects that are in our VALID list
    final_aspects = []
    for aspect in extracted_aspects:
        # Check if the lowercase aspect (or singular version) is in our list
        # This allows "pizzas" to pass if "pizza" is in the list
        norm_aspect = aspect.lower()
        if norm_aspect in VALID_RESTAURANT_TERMS or norm_aspect.rstrip('s') in VALID_RESTAURANT_TERMS:
            final_aspects.append(aspect)

    return final_aspects

def get_audio_emotion(audio_path):
    speech, sr = librosa.load(audio_path, sr=16000)
    inputs = ser_processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = ser_model(**inputs).logits

    pred_id = torch.argmax(logits, dim=-1).item()
    return ser_model.config.id2label[pred_id]

def run_full_pipeline(audio_path):
    print(f"\n--- Processing: {audio_path.split('/')[-1]} ---")

    emotion = get_audio_emotion(audio_path)
    print(f"🔊 Detected Vocal Emotion: {emotion}")

    print("📝 Transcribing audio...")
    transcription = asr_model.transcribe(audio_path)["text"].strip()
    print(f"   Transcript: \"{transcription}\"")

    # Extract AND Filter
    aspects_found = extract_aspects_custom(transcription, absa_model, absa_tokenizer)

    print("-" * 30)
    if not aspects_found:
        print(f"🚀 FINAL OUTPUT: Customer is {emotion}, but no relevant food aspect was found.")
    else:
        aspect_string = ", ".join(aspects_found)
        print(f"🧠 Detected Aspects: {aspects_found}")
        print(f"🚀 FINAL OUTPUT: Customer is {emotion} about: {aspect_string}")
    print("-" * 30)

In [13]:


# REPLACE with your actual test audio file path
test_file = "/content/drive/MyDrive/dlProject/Datasets/moiz.wav"

# Check if file exists before running
import os
if os.path.exists(test_file):
    run_full_pipeline(test_file)
else:
    print(f"File not found: {test_file}")
    print("Please upload a .wav file to your Drive and update the 'test_file' path.")


--- Processing: moiz.wav ---
🔊 Detected Vocal Emotion: angry
📝 Transcribing audio...
   Transcript: "Your food is disgusting!"
------------------------------
🧠 Detected Aspects: ['food']
🚀 FINAL OUTPUT: Customer is angry about: food
------------------------------


In [14]:


# REPLACE with your actual test audio file path
test_file = "/content/drive/MyDrive/dlProject/Datasets/ahad.wav"

# Check if file exists before running
import os
if os.path.exists(test_file):
    run_full_pipeline(test_file)
else:
    print(f"File not found: {test_file}")
    print("Please upload a .wav file to your Drive and update the 'test_file' path.")


--- Processing: ahad.wav ---
🔊 Detected Vocal Emotion: calm
📝 Transcribing audio...
   Transcript: "Oh that's good. You sound like a monster."
------------------------------
🚀 FINAL OUTPUT: Customer is calm, but no relevant food aspect was found.
------------------------------


In [ ]:
import os
from IPython.display import Audio, display
display(Audio(test_file))